#### 2023-09-19(화)헬스클럽 리뷰 

In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd  # pandas 라이브러리를 import합니다.
import pymongo as mg
import os
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [14]:
browser = webdriver.Chrome(executable_path='../../../../chromedriver.exe')

In [15]:
browser.implicitly_wait(10)
url = 'https://play.google.com/store/apps/details?id=com.supplies.gymboxx.userclient&utm_source=mms&utm_medium=mms&utm_campaign=app_renewal'
browser.get(url)

In [16]:
browser.implicitly_wait(10)

In [17]:
click_path = ' div > div.Jwxk6d > div:nth-child(5) > div > div > button > span'
browser.find_element_by_css_selector(click_path).click()

In [18]:
  # 리뷰 정보 수집
service_name = browser.find_element_by_css_selector('div.fysCi > div > div:nth-child(2) > div:nth-child(1) > header > div.YNR7H > div.gSGphe > div').text
stars = browser.find_element_by_css_selector('div.fysCi > div > div:nth-child(2) > div:nth-child(1) > header > div.Jx4nYe > div').get_attribute('aria-label')
                # 리뷰 텍스트 내용 가져오기
review_text = browser.find_element_by_css_selector('div.fysCi > div > div:nth-child(2) > div:nth-child(1) > div.h3YV2d').text
print(service_name),print(stars),print(review_text)

김국빈
별표 5개 만점에 5개를 받았습니다.
짐박스 이용자로서 정말 편하게 잘 사용하고 있습니다. 한 가지 건의사항으로, 다른 지점으로 출석할 시에 지오더 지점도 같이 자동으로 변경되면 어떨까 합니다. 간혹 다른 지점에 출석하고 나서 지오더 지점을 변경하는 걸 깜빡하고 주문하는 경우가 있었는데, 직원분께서 빠르게 주문 수정해주셨지만 지오더 지점도 같이 변경되면 좀 더 편할 것 같아요.


(None, None, None)

In [19]:
from selenium.webdriver.common.keys import Keys
# page down
# browser.find_element_by_css_selector('body').send_keys(Keys.PAGE_DOWN)

In [20]:
browser.find_element_by_css_selector('body').send_keys(Keys.PAGE_DOWN)

In [21]:
client = mg.MongoClient(host='mongodb://localhost:27017')
database = client['project_data_analytics']
collection = database['reviews']


# 전체 리뷰 가져오기
serviceapp_columns_name = ['service_name', 'stars', 'reviews']
reviews_list = list()

itemlist= browser.find_element(By.XPATH, '//*[@id="yDmH0d"]/div[4]/div[2]/div/div/div/div/div[2]/div')
for i in range(1,517):
    try:
      
        # WebDriverWait를 사용하여 요소를 찾고 대기
        WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'header > div.YNR7H > div.gSGphe > div')))
       
        service_name = browser.find_element(By.XPATH, '//*[@id="yDmH0d"]/div[4]/div[2]/div/div/div/div/div[2]/div/div[2]/div[{}]/header/div[1]/div[1]/div'.format(i))
                # 리뷰 정보 수집
        #service_name = browser.find_element_by_css_selector('div.fysCi > div > div:nth-child(2) > div:nth-child({0}) > header > div.YNR7H > div.gSGphe > div'.format(i)).text
        stars = browser.find_element_by_css_selector(' div.fysCi > div > div:nth-child(2) > div:nth-child({0}) > header > div.Jx4nYe > div'.format(i)).get_attribute('aria-label')
            # 리뷰 텍스트 내용 가져오기
        review_text = browser.find_element_by_css_selector(' div.fysCi > div > div:nth-child(2) > div:nth-child({0}) > div.h3YV2d'.format(i)).text
        browser.find_element_by_css_selector('body').send_keys(Keys.PAGE_DOWN)
        review_list = [service_name.text, stars, review_text]
        reviews_list.append(review_list)
       
        df_reviews = pd.DataFrame(data=reviews_list, columns=serviceapp_columns_name)
        data_dict = df_reviews.to_dict(orient='records')
        collection.insert_many(data_dict)
        print(i)
        browser.execute_script("arguments[0].scrollIntoView();", service_name)    
           
   
    except Exception as e:
        print(f'Error: {e}')
        pass

      
    
print('service_name : {}, review count : {}'.format(service_name, len(reviews_list)))
browser.back() # 리스트로 돌아가기
time.sleep(3)
print('Done')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=116.0.5845.188)
